In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
import nibabel as nib
import traceback
import joblib

import registration as rg
from scipy.ndimage import affine_transform

from registration.utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
from registration.reg_utils import loadNiiImages
from registration.vol2affine import vol2affine

In [5]:
templateImagePath = "../CCF_DATA/average_template_10.nii.gz"
dataImagePath = "../../registration/B39/brain_10.nii.gz"
fixedImagePath = templateImagePath
movingImagePath = dataImagePath

In [6]:
fImage  = nib.load(fixedImagePath)
mImage = nib.load(movingImagePath)

In [8]:
mdata = mImage.get_fdata()

In [26]:
fdata, mdata  = loadNiiImages([fixedImagePath, movingImagePath])
R, r1, r2 = vol2affine(moving=mdata, template=fdata,pivot=(0, 0, 0))

100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:19<00:00, 69.39it/s]


In [28]:
R

array([[ 9.99841133e-01,  2.39173003e-04,  1.78227897e-02,
         0.00000000e+00],
       [ 1.80780979e-04,  9.99722469e-01, -2.35574167e-02,
         0.00000000e+00],
       [-1.78234776e-02,  2.35568963e-02,  9.99563603e-01,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [8]:
fixedImagePath = templateImagePath
movingImagePath = dataImagePath
A, axisAlignedData = rg.axisAlignData(fixedImagePath, movingImagePath)
create_nifti_image(alignedData, 1, "axisAlign10_0.nii.gz",1)

100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:20<00:00, 64.72it/s]


In [11]:
A

array([[ 9.99841133e-01,  2.39173003e-04,  1.78227897e-02,
        -1.26197062e+01],
       [ 1.80780979e-04,  9.99722469e-01, -2.35574167e-02,
         1.66632515e+01],
       [-1.78234776e-02,  2.35568963e-02,  9.99563603e-01,
        -1.00368708e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [32]:
fixed, moving  = loadNiiImages([fixedImagePath, movingImagePath])

In [ ]:
templateImagePath = "../CCF_DATA/average_template_25.nii.gz"
dataImagePath = "../../registration/B39/brain_25.nii.gz"
fixedImagePath = templateImagePath
movingImagePath = dataImagePath
fdata, mdata  = loadNiiImages([fixedImagePath, movingImagePath])
R, r1, r2 = vol2affine(moving=mdata, template=fdata,pivot=(0, 0, 0))

In [33]:
origin = np.array(moving.shape)/2

A1 = np.eye(4)
A1[0:3, 3] = -origin

A2 = np.eye(4)
A2[0:3,0:3] = R[0:3,0:3]

A3 = np.eye(4)
A3[0:3, 3] = origin
A = (A3@A2)@A1

In [36]:
alignedData = affine_transform(moving,np.linalg.inv(A), output_shape = moving.shape, order =1)

In [37]:
create_nifti_image(alignedData, 1, "axisAlign10.nii.gz",1)